In [ ]:
from local.torch_basics import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.all import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *
from local.metrics import *
from local.text.core import *
from local.text.data import *
from local.text.models.core import *
from local.text.models.awdlstm import *
from local.text.learner import *
from local.callback.rnn import *
from local.callback.all import *

# Transfer learning in text

> How to fine-tune a language model and train a classifier

## Finetune a pretrained Language Model

First we get our data and tokenize it.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')
df_tok,count = tokenize_df(df, 'text')
texts,lens = df_tok['text'],df_tok['text_lengths'].values.astype(np.int)

Then we put it in a `DataSource`. For a language model, we don't have targets, so there is only one transform to numericalize the texts. Note that `tokenize_df` returns the count of the words in the corpus to make it easy to create a vocabulary.

In [ ]:
splits = RandomSplitter()(texts)
vocab = make_vocab(count)
dsrc = DataSource(texts, [Numericalize(vocab)], filts=splits)

Then we use that `DataSource` to create a `DataBunch`. Here the class of `TfmdDL` we need to use is `LMDataLoader` which will concatenate all the texts in a source (with a shuffle at each epoch for the training set), split it in `bs` chunks then read continuously through it.

In [ ]:
dbunch = LMDataLoader.dbunchify(dsrc, lens=lens, bs=64, seq_len=72)

In [ ]:
dbunch.show_batch()

,text
0,"xxbos xxmaj fair drama / love story movie that focuses on the lives of blue collar people finding new life thru new xxunk acting here is good but the film fails in cinematography , screenplay , directing and xxunk story / script is only average at xxunk film will be enjoyed by xxmaj fonda and xxmaj de xxmaj niro fans and by people who love middle age love stories where in the"
1,"strictly xxunk . a must see . xxbos a complete waste of time \n\n xxmaj xxunk xxmaj xxunk is a complete waste of time . xxmaj the script and dialogues are poorly written , the direction is xxunk and the acting borders on xxunk movie was clearly aiming for the xxmaj xxunk xxmaj de xxmaj xxunk crowd but it falls far short of the mark because it does not have even one"
2,"pay you enough . "" xxbos xxmaj this movie was probably the worst movie i have ever seen . xxmaj here are the things that immediately jump out at me : 1 . xxmaj the woods were more like hills in xxmaj los xxmaj angeles with a couple trees and xxunk . xxmaj not scary whatsoever . xxmaj news flash , if you are filming in the xxmaj southern xxmaj california area"
3,"movie him and her are working together to take down the bad guys and every time they exchange words it just feels like the cheap lines given before a sex scene in a porn movie . xxmaj horrible . xxmaj do n't waste your time . xxbos xxmaj for the big xxunk among us , "" the xxmaj intruder "" is a xxunk incoherent movie from xxmaj france that gives so -"
4,"south ; not because you might witness a murder , but you may run across a xxunk plantation . ) xxmaj this is the point where the story , not the movie , actually comes close to being good . \n\n xxmaj while xxmaj tony xxmaj march will never have to practice his xxmaj oscar speech , his xxmaj sheriff xxmaj dean becomes a creepy xxunk of a normal guy torn by"
5,bored with his career ; he xxunk agrees to play xxmaj othello . xxmaj he gets deep into character as a jealous and murderous man . xxmaj he begins walking a thin line between xxunk and reality and ends up confusing his role with his own life and eventually kills his xxunk xxmaj xxunk has no memory of the xxunk xxunk . \n\n xxmaj colman seems xxunk in this role . xxmaj
6,"these people can afford to live in a nice xxmaj manhattan xxunk considering what they do for a living ? xxup xxunk just loves to insult the viewer 's intelligence , even if they 're just around xxmaj xxunk 's level . i know a person who makes $ 100 , xxrep 3 0 a year as a web designer and lives in a xxunk one - bedroom apartment in xxmaj manhattan"
7,"dead "" ; "" xxunk , bring me the big knife "" , "" who 's dead "" , "" do you love him xxmaj loretta … .. , good because when you do , they drive you crazy because they know they can "" . i always put xxmaj moonstruck on when there 's nothing good to watch because it makes me happy . xxbos xxmaj one more of those brilliant"
8,"\n\n xxmaj the heavy metal score -- was someone making a pun ? -- is , at times , xxunk annoying , but the cinematography by xxmaj frank xxmaj stanley is knockout , particularly the xxunk scenes . xxbos xxmaj the name ( xxunk ) of the main character is the xxmaj german word for "" woman "" . i do n't know if that was intentional or not , but if"
9,", but that does n't mean it 's high quality . xxmaj yes , millions of xxmaj xxunk fans xxup can be wrong . \n\n xxmaj i 'll be the first to admit , that part of the problem for me was the xxunk - hype over the film . i waited a month or so to see it and ultimately , it did n't live up to the expectations set upon"


Then we have a convenience method to directly grab a `Learner` from it, using the `AWD_LSTM` architecture.

In [ ]:
learn = language_model_learner(dbunch, AWD_LSTM, vocab, metrics=[accuracy, Perplexity()], path=path)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-2, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.800079,4.339657,0.258123,76.681259,00:04
1,4.758967,4.313660,0.259611,74.713448,00:04
2,4.740184,4.302836,0.260510,73.909134,00:04
3,4.739078,4.301117,0.260835,73.782196,00:04


Once we have fine-tuned the pretrained language model to this corpus, we save the encoder since we will use it for the classifier.

In [ ]:
learn.save_encoder('enc')

## Use it to train a classifier

For classification, we need to use two set of transforms: one to numericalize the texts and the other to encode the labels as categories.

In [ ]:
splits = RandomSplitter()(range_of(df_tok))
dsrc = DataSource(df_tok, filts=splits, tfms=[
    [attrgetter("text"), Numericalize(vocab)],
    [attrgetter("label"), Categorize()]])

We opnce again use a subclass of `TfmdDL` for the dataloaders, since we want to sort the texts (sortish for the training set) by order of lengths. We also use `pad_collate` to create batches form texts of different lengths.

In [ ]:
trn_dl = SortedDL(dsrc.train, create_batch=pad_collate, after_batch=[Cuda], shuffle=True, drop_last=True)
val_dl = SortedDL(dsrc.valid, create_batch=pad_collate, after_batch=[Cuda])
dbunch = DataBunch(trn_dl, val_dl)

In [ ]:
dbunch.show_batch(max_n=2)

Then we once again have a convenience function to create a classifier from this `DataBunch` with the `AWD_LSTM` architecture.

In [ ]:
learn = text_classifier_learner(dbunch, AWD_LSTM, vocab, metrics=[accuracy], path=path)

We load our pretrained encoder.

In [ ]:
learn = learn.load_encoder('enc')

Then we can train with gradual unfreezing and differential learning rates.

In [ ]:
learn.fit_one_cycle(4, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.849841,0.601815,0.675000,00:06
1,0.753493,0.568468,0.720000,00:06
2,0.753582,0.548259,0.745000,00:06
3,0.740121,0.533096,0.745000,00:06


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(8, np.array([1e-5,3e-4,1e-4,3e-3,1e-3]), moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.710222,0.536359,0.750000,00:10
1,0.715015,0.531864,0.745000,00:10
2,0.737472,0.531057,0.750000,00:09
3,0.682374,0.526255,0.745000,00:10
4,0.707951,0.531820,0.735000,00:10
5,0.723326,0.524388,0.760000,00:10
6,0.685744,0.527304,0.745000,00:10
7,0.732956,0.520143,0.755000,00:10
